# Setup

```bash
conda create -n giswqs_geospatial_20250622 python=3.11
conda activate giswqs_geospatial_20250622
conda install -c conda-forge mamba
mamba install -c conda-forge geospatial
mamba install -c conda-forge python-duckdb duckdb-engine jupysql leafmap
```

In [1]:
import duckdb
import pandas as pd
import os
import geopandas as gpd

In [2]:
# con.sql("SHOW TABLES")

# First look at the data

## Woreda points data

In [3]:
con = duckdb.connect("woredo.db")

In [ ]:
con.sql("SHOW TABLES")

In [5]:
con.install_extension("spatial")
con.load_extension("spatial")

In [6]:
indir = "G:/ales_guyo/guyo_woreda/from_gee_cleaned"

In [ ]:
os.listdir(indir)

In [19]:
infile_csv = os.path.join(indir, 'data_25km_long_format_lonlat.csv')

woreda_points_csv_file = infile_csv

In [ ]:
con.sql(f"SELECT COUNT(*) FROM '{infile_csv}'")

In [15]:
woreda_points_3 = con.sql(f"SELECT * FROM '{infile_csv}' LIMIT 3").df()

In [ ]:
woreda_points_3

In [ ]:
woreda_points_3.columns

In [ ]:
woreda_points_3.Geographic

### Convert to parquet

In [7]:
woreda_points_parquet_file = 'data_25km_long_format_lonlat.parquet'

In [ ]:
con.sql(f"SELECT COUNT(*) FROM '{woreda_points_csv_file}'")

In [ ]:
con.sql(f"FROM '{woreda_points_csv_file}' LIMIT 1").df().columns[0:10]

In [ ]:
# Convert CSV with lat/lon to parquet with geometry column
con.sql(f"""
    COPY (
        SELECT 
            *,
            ST_Point(latitude, longitude) as geometry 
        FROM '{woreda_points_csv_file}'
    ) TO '{woreda_points_parquet_file}' (FORMAT PARQUET)
""")

In [ ]:
con.sql(f"SELECT COUNT(*) FROM '{woreda_points_parquet_file}'")

In [ ]:
con.sql(f"SELECT latitude, longitude, geometry FROM '{woreda_points_parquet_file}' LIMIT 2")

In [ ]:
con.sql(f"SELECT * FROM '{woreda_points_parquet_file}' LIMIT 3")

### Store as table in the database

In [ ]:
## First, test different approaches for small subset of data
## do i need to format the geometry column ?


# con.sql(f"""
#     SELECT * EXCLUDE geometry, ST_GeomFromWKB(geometry) AS geometry 
#     FROM '{woreda_points_parquet_file}'
#     LIMIT 3
# """).df()


# con.sql(f"""
#     SELECT * EXCLUDE geometry, ST_GeomFromText(ST_AsText(geometry)) AS geometry 
#     FROM '{woreda_points_parquet_file}'
#     LIMIT 3
# """).df()

# con.sql(f"""
#     SELECT * EXCLUDE geometry, ST_AsText(geometry) AS geometry  -- this one works when .df() , but varchar in database
#     FROM '{woreda_points_parquet_file}'
#     LIMIT 3
# """).df()

con.sql(f"""
    SELECT *  -- this one doesn't work when .df(), but is geometry in databse -- so use this one for storing table in database
    FROM '{woreda_points_parquet_file}'
    LIMIT 3
""")


In [ ]:
## Now do for all data (not done cause too slow...)

# con.sql(f"""
#     CREATE OR REPLACE TABLE woreda_points AS
#     SELECT *  
#     FROM '{woreda_points_parquet_file}'
# """)

## a bit too slow...

### Reproject to UTM, store as another table in database

In [ ]:
## with table in database

# con.sql(f"""
# CREATE OR REPLACE TABLE woreda_points_utm AS
# SELECT * EXCLUDE geometry, ST_Transform(ST_GeomFromText(ST_AsText(geometry)), 'EPSG:4326', '{utm_code}', true) AS geometry 
# FROM woreda_points 
# """)

In [ ]:
### with external .gpkg file

woreda_points_parquet_utm_file = 'woreda_points_utm.parquet'

con.sql(f"""
COPY (
    SELECT * EXCLUDE geometry, ST_Transform(geometry, 'EPSG:4326', 'EPSG:32637', true) AS geometry 
    FROM '{woreda_points_parquet_file}' 
) TO '{woreda_points_parquet_utm_file}' (FORMAT PARQUET)
""")

## 32m

## Other woreda data

In [8]:
# infile_csv_woredas = os.path.join(indir, 'woredas_25km_cleaned.csv')

In [11]:
# con.sql(f"SELECT COUNT(*) FROM '{infile_csv_woredas}'")

In [12]:
# con.sql(f"SELECT * FROM '{infile_csv}' LIMIT 3")

In [10]:
# con.sql(f"SELECT * FROM '{infile_csv_woredas}' LIMIT 3")

In [13]:
# con.sql(f"SELECT COUNT(DISTINCT name_1_3) FROM '{infile_csv}'")

In [14]:
# con.sql(f"SELECT COUNT(DISTINCT name) FROM '{infile_csv_woredas}'")

In [ ]:
# need the coordinates not yet here...

In [ ]:
# con.sql("COPY (SELECT * FROM )) TO 'cities.parquet' (FORMAT PARQUET)")

## My old data

In [9]:
old_file_csv = "D:/hadi/projects/alessandro_guyo/data_processed/rasterToPoints_merged/allRasters_stack_pts_df_onlyXY_joined_subcatchments.csv"

In [ ]:
con.sql(f"SELECT COUNT(*) FROM '{old_file_csv}'")

In [ ]:
con.sql(f"SELECT * FROM '{old_file_csv}' LIMIT 3")

In [ ]:
con.sql(f"SELECT COUNT(DISTINCT (X, Y)) FROM '{old_file_csv}'")


In [7]:
old_file_parquet = "old_points_with_geometry.parquet"

In [ ]:
# Convert CSV with lat/lon to parquet with geometry column
con.sql(f"""
    COPY (
        SELECT 
            *,
            ST_Point(X, Y) as geometry 
        FROM '{old_file_csv}'
    ) TO '{old_file_parquet}' (FORMAT PARQUET)
""")


In [ ]:
con.sql(f"FROM '{old_file_parquet}'")

In [ ]:
con.sql(f"SELECT COUNT(*) FROM '{old_file_parquet}'")

In [ ]:
con.sql(f"SELECT * FROM '{old_file_parquet}' LIMIT 3")

In [ ]:
## Check geometry column by converting to geopandas and plotting them

con.sql(f"SELECT * EXCLUDE geometry, ST_GeomFromWKB(geometry) AS geometry FROM '{old_file_parquet}' LIMIT 3").df()


In [ ]:
con.sql(f"SELECT * EXCLUDE geometry, ST_AsText(geometry) AS geometry FROM '{old_file_parquet}' LIMIT 3").df()


In [ ]:
temp = con.sql(f"SELECT * EXCLUDE geometry, ST_AsText(geometry) AS geometry FROM '{old_file_parquet}' LIMIT 3").df()
# temp = con.sql(f"SELECT * FROM '{old_file_parquet}' LIMIT 3").df()

temp

In [15]:
from shapely.wkt import loads

temp['geometry'] = temp['geometry'].apply(loads)

In [ ]:
## convert pandas df to geopandas gdf, using geometry column
temp_gdf = gpd.GeoDataFrame(temp, geometry='geometry', crs='EPSG:4326')

temp_gdf.explore()

In [ ]:
# con.sql(f"""
#     CREATE OR REPLACE TABLE old_points AS
#     SELECT * EXCLUDE geometry, ST_GeomFromText(ST_AsText(geometry)) AS geometry
#     FROM '{old_file_parquet}'
# """)


con.sql(f"""
    CREATE OR REPLACE TABLE old_points AS
    SELECT * 
    FROM '{old_file_parquet}'
""")

In [ ]:
con.sql("FROM old_points")

## Boundary data

### just example use old data

In [19]:
## just example use old data

old_boundary_file = "D:/hadi/projects/alessandro_guyo/data/afar_somali_shape_sub_watershed_cropped.gpkg"

In [20]:
old_boundary = gpd.read_file(old_boundary_file)

In [ ]:
old_boundary

In [21]:
old_boundary_parquet_file = "old_boundary.parquet"

con.sql(
    f"""
COPY (
    SELECT *
    FROM ST_Read('{old_boundary_file}')
) TO '{old_boundary_parquet_file}'(FORMAT PARQUET)
"""
)


In [ ]:
con.sql(f"SELECT * FROM '{old_boundary_parquet_file}' LIMIT 3")

In [ ]:
con.sql("SELECT * FROM 'old_boundary.parquet' LIMIT 3").df().columns

In [24]:
# con.sql(
#     """
# CREATE OR REPLACE TABLE old_boundary AS
# SELECT * EXCLUDE geom, ST_GeomFromText(ST_AsText(geom))
# AS geometry FROM 'old_boundary.parquet'
# """
# )


con.sql(
    """
CREATE OR REPLACE TABLE old_boundary AS
SELECT * 
FROM 'old_boundary.parquet'
"""
)

In [ ]:
con.sql("FROM old_points")

In [ ]:
con.sql("FROM old_boundary")

### Woreda polygons

In [27]:
woreda_polygons_file =  "D:/hadi/projects/guyo/merged.gpkg"


In [28]:
con.sql(
    f"""
COPY (
    SELECT *
    FROM ST_Read('{woreda_polygons_file}')
) TO 'woreda_polygons.parquet' (FORMAT PARQUET)
"""
)

In [29]:
con.sql(
    """
CREATE OR REPLACE TABLE woreda_polygons AS
SELECT * 
FROM 'woreda_polygons.parquet'
"""
)

### Woreda border segments (lines) with woreda-pair id

In [30]:
woreda_border_segments_file = "G:/ales_guyo/guyo_woreda/output/border_segments/boundary_segments_method2_dissolved.gpkg"

In [31]:
con.sql(
    f"""
COPY (
    SELECT *
    FROM ST_Read('{woreda_border_segments_file}')
) TO 'woreda_border_segments.parquet' (FORMAT PARQUET)
"""
)

In [32]:
con.sql(
    """
CREATE OR REPLACE TABLE woreda_border_segments AS
SELECT * 
FROM 'woreda_border_segments.parquet'
"""
)

In [ ]:
con.sql("FROM woreda_border_segments LIMIT 3")

In [ ]:
con.sql("FROM woreda_border_segments LIMIT 3").df().columns

## Test spatial join if the geometry columns are fine

In [ ]:
con.sql("SHOW TABLES")

In [ ]:
con.sql("FROM old_points LIMIT 3")

In [ ]:
con.sql("FROM woreda_polygons LIMIT 3")

In [ ]:
con.sql("FROM woreda_polygons LIMIT 3").df().columns

### old_points within a selected woreda_polygons with treated_not_treated == 1.0

In [57]:
selected_woreda_polygons_df = con.sql("""
SELECT * EXCLUDE geom, ST_AsText(geom) AS geom 
FROM woreda_polygons 
WHERE treated_not_treated = 1.0
""").df()

In [ ]:
selected_woreda_polygons_df.head(n=1)

In [61]:
selected_woreda_polygons_df['geom'] = selected_woreda_polygons_df['geom'].apply(loads)

In [62]:
selected_woreda_polygons_gdf = gpd.GeoDataFrame(selected_woreda_polygons_df, geometry='geom', crs='EPSG:4326')

In [ ]:
selected_woreda_polygons_gdf.explore()

In [64]:
## Create table of selected woreda polygons

con.sql("""
CREATE OR REPLACE TABLE selected_woreda_polygons AS
SELECT * EXCLUDE geom, ST_AsText(geom) AS geom 
FROM woreda_polygons 
WHERE treated_not_treated = 1.0
""")

In [65]:
## Reproject selected_woreda_polygons table to UTM 37N

utm_code = 'EPSG:32637'

con.sql(f"""
CREATE OR REPLACE TABLE selected_woreda_polygons_utm AS
SELECT * EXCLUDE geom, ST_Transform(ST_GeomFromText(geom), 'EPSG:4326', '{utm_code}', true) AS geometry 
FROM selected_woreda_polygons 
""")

In [ ]:
con.sql("FROM selected_woreda_polygons_utm LIMIT 3")

In [ ]:
con.sql("SELECT COUNT(*) FROM old_points")

##### Reproject old_points table to UTM 37N

In [ ]:
## Reproject old_points table to UTM 37N

con.sql(f"""
CREATE OR REPLACE TABLE old_points_utm AS
SELECT * EXCLUDE geometry, ST_Transform(ST_GeomFromText(ST_AsText(geometry)), 'EPSG:4326', '{utm_code}', true) AS geometry 
FROM old_points 
""")

In [ ]:
con.sql("FROM old_points_utm LIMIT 3")

#### Add custom point id

In [ ]:
## Add custom_id as 1 to number of rows

# con.sql("""
# CREATE OR REPLACE TABLE old_points_utm AS
# SELECT 
#     *,
#     ROW_NUMBER() OVER () as custom_points_id
# FROM old_points_utm;
# """)


## a better way!
con.sql("""
CREATE OR REPLACE TABLE old_points_utm_temp AS
SELECT 
    *,
    ROW_NUMBER() OVER () as custom_points_id
FROM old_points_utm;
""")

con.sql("""
CREATE OR REPLACE TABLE old_points_utm AS
SELECT * FROM old_points_utm_temp;
""")

In [ ]:
con.sql("FROM old_points_utm LIMIT 3")

In [79]:
# con.sql("""
# ALTER TABLE old_points_utm DROP COLUMN custom_points_id;
# """)

In [ ]:
con.sql("FROM old_points_utm LIMIT 3")

In [81]:
con.sql("ALTER TABLE old_points_utm RENAME COLUMN custom_points_id_1 TO custom_points_id;")


In [ ]:
con.sql("FROM old_points_utm LIMIT 3")

#### Add custom woreda polygon id

In [83]:
## Add custom_id as 1 to number of rows

# con.sql("""
# CREATE OR REPLACE TABLE selected_woreda_polygons_utm AS
# SELECT 
#     *,
#     ROW_NUMBER() OVER () as custom_woreda_id
# FROM selected_woreda_polygons_utm;
# """)

## a better way!
con.sql("""
CREATE OR REPLACE TABLE selected_woreda_polygons_utm_temp AS
SELECT 
    *,
    ROW_NUMBER() OVER () as custom_woreda_id
FROM selected_woreda_polygons_utm;
""")

con.sql("""
CREATE OR REPLACE TABLE selected_woreda_polygons_utm AS
SELECT * FROM selected_woreda_polygons_utm_temp;
""")

In [ ]:
con.sql("SELECT custom_woreda_id, geometry FROM selected_woreda_polygons_utm LIMIT 3")

In [ ]:
con.sql("SELECT COUNT(*) FROM old_points")

#### Test spatial join

In [ ]:
## Spatial join

test_join_df = con.sql("""
SELECT
    ST_AsText(points.geometry) AS points_geometry,
    points.custom_points_id,
    woreda.custom_woreda_id
FROM selected_woreda_polygons_utm as woreda
JOIN old_points_utm as points
ON ST_Intersects(woreda.geometry, points.geometry)
""").df()

test_join_df

In [92]:
test_join_df['points_geometry'] = test_join_df['points_geometry'].apply(loads)

In [ ]:
test_join_gdf = gpd.GeoDataFrame(test_join_df, geometry='points_geometry', crs='EPSG:32637')

In [ ]:
selected_woreda_polygons_utm_df = con.sql("SELECT * EXCLUDE geometry, ST_AsText(geometry) AS geometry FROM selected_woreda_polygons_utm").df()

selected_woreda_polygons_utm_df.head(n=2)

In [101]:
selected_woreda_polygons_utm_df['geometry'] = selected_woreda_polygons_utm_df['geometry'].apply(loads)

In [103]:
selected_woreda_polygons_utm_gdf = gpd.GeoDataFrame(selected_woreda_polygons_utm_df, geometry='geometry', crs='EPSG:32637')

In [108]:
# import folium

# m = folium.Map()

# selected_woreda_polygons_utm_gdf.explore(
#     m=m,
#     color='red',
#     fillOpacity=0.3,
#     name='Woreda Polygons',
#     show=True
# )

# test_join_gdf.explore(
#     m=m,
#     color='blue',
#     marker_kwds={'radius': 5},
#     name='Test Join Points',
#     show=True
# )

# folium.LayerControl().add_to(m)

# m


## slow to pan...


In [ ]:
# !pip install lonboard

In [ ]:
# import lonboard


# # Try different layer creation methods
# try:
#     # Newer version
#     woreda_layer = lonboard.Layer.from_geopandas(
#         selected_woreda_polygons_utm_gdf,
#         get_fill_color=[255, 0, 0, 50]
#     )
# except AttributeError:
#     # Older version
#     woreda_layer = lonboard.ScatterplotLayer.from_geopandas(
#         selected_woreda_polygons_utm_gdf,
#         get_fill_color=[255, 0, 0, 50]
#     )

# try:
#     points_layer = lonboard.Layer.from_geopandas(
#         test_join_gdf,
#         get_fill_color=[0, 0, 255, 255]
#     )
# except AttributeError:
#     points_layer = lonboard.ScatterplotLayer.from_geopandas(
#         test_join_gdf,
#         get_fill_color=[0, 0, 255, 255]
#     )

# map_view = lonboard.Map([woreda_layer, points_layer])
# map_view

In [115]:
## just save as .gpkg and inspect in QGIS

selected_woreda_polygons_utm_gdf.to_file("selected_woreda_polygons_utm.gpkg", driver="GPKG")
test_join_gdf.to_file("test_join_gdf.gpkg", driver="GPKG")



Checked in QGIS, looks correct!

## Now calculate distance

### Just example using old points data and woreda border segments

In [ ]:
con.sql("FROM old_points_utm LIMIT 3")

#### Reproject woreda_border_segments to utm_code

In [ ]:
## Reproject woreda_border_segments to utm_code

con.sql(f"""
CREATE OR REPLACE TABLE woreda_border_segments_utm AS
SELECT * EXCLUDE geom, ST_Transform(ST_GeomFromText(ST_AsText(geom)), 'EPSG:4326', '{utm_code}', true) AS geometry 
FROM woreda_border_segments 
""")

In [ ]:
con.sql("""
SELECT * FROM woreda_border_segments_utm LIMIT 3
""")

#### Distance between every point and every line (nope)

In [ ]:
# con.sql("""
# CREATE TABLE distances_old_points_to_woreda_border_segments_v1 AS
# SELECT
#   p.custom_points_id,
#   b.polygon_pairs,
#   ST_Distance(p.geometry, b.geometry) AS distance_meters
# FROM old_points_utm p
# CROSS JOIN woreda_border_segments_utm b;
# """)

## no, this is distance from every point, to every segment line!!
## killed

In [ ]:
# con.sql("SHOW TABLES")

#### Find closest border only 

##### Option 1

In [ ]:
# CREATE TABLE distances_old_points_to_woreda_border_segments_v1 AS
# SELECT
#   p.custom_points_id,
#   b.polygon_pairs,
#   ST_Distance(p.geometry, b.geometry) AS distance_meters
# FROM old_points_utm p
# CROSS JOIN woreda_border_segments_utm b
# WHERE (p.custom_points_id, ST_Distance(p.geometry, b.geometry)) IN (
#   SELECT 
#     p2.custom_points_id,
#     MIN(ST_Distance(p2.geometry, b2.geometry))
#   FROM old_points_utm p2
#   CROSS JOIN woreda_border_segments_utm b2
#   GROUP BY p2.custom_points_id
# );

##### Option 2: Using window functions

In [ ]:
# CREATE TABLE distances_old_points_to_closest_border AS
# SELECT 
#   custom_points_id,
#   polygon_pairs,
#   distance_meters
# FROM (
#   SELECT
#     p.custom_points_id,
#     b.polygon_pairs,
#     ST_Distance(p.geometry, b.geometry) AS distance_meters,
#     ROW_NUMBER() OVER (PARTITION BY p.custom_points_id ORDER BY ST_Distance(p.geometry, b.geometry)) as rn
#   FROM old_points_utm p
#   CROSS JOIN woreda_border_segments_utm b
# ) ranked
# WHERE rn = 1;

##### Option 3: Using spatial filtering first (most efficient for large datasets) - Claude recommends


In [ ]:
# con.sql("""
# CREATE TABLE distances_old_points_to_closest_border AS
# SELECT 
#   custom_points_id,
#   polygon_pairs,
#   distance_meters
# FROM (
#   SELECT
#     p.custom_points_id,
#     b.polygon_pairs,
#     ST_Distance(p.geometry, b.geometry) AS distance_meters,
#     ROW_NUMBER() OVER (PARTITION BY p.custom_points_id ORDER BY ST_Distance(p.geometry, b.geometry)) as rn
#   FROM old_points_utm p
#   CROSS JOIN woreda_border_segments_utm b
#   WHERE ST_DWithin(p.geometry, b.geometry, 25000)  -- Only check within 25km
# ) ranked
# WHERE rn = 1
# """)

## Likely will take over 240 minutes...
## killed

### ** Real woredo points data, distance to border segments between individual woreda **

In [ ]:
con.sql(f"FROM '{woreda_points_parquet_utm_file}' LIMIT 3")

In [ ]:
con.sql("SHOW TABLES")

In [ ]:
con.sql("FROM woreda_polygons LIMIT 3")

In [ ]:
con.sql("FROM woreda_border_segments_utm LIMIT 3")

In [ ]:
con.sql("FROM '{woreda_points_parquet_utm_file}' LIMIT 3").df().columns

In [ ]:
con.sql("FROM woreda_border_segments_utm LIMIT 3").df().columns

In [15]:
woreda_polygons_addRowId_file = "woreda_addRowId.gpkg"

In [ ]:
con.sql(f"FROM '{woreda_polygons_addRowId_file}' LIMIT 3")  ## custom_woreda_id

In [17]:
# Read the GeoPackage file and register it as a table
con.sql(f"""
CREATE OR REPLACE TABLE woreda_polygons_addRowId AS 
SELECT * FROM ST_Read('{woreda_polygons_addRowId_file}')
""")

In [ ]:
con.sql("FROM woreda_polygons_addRowId LIMIT 3")  ## custom_woreda_id

In [ ]:
columns_info = con.sql(f"DESCRIBE '{woreda_points_parquet_utm_file}'").df()
print("Available columns:", columns_info['column_name'].tolist())

#### Test run

In [ ]:
## Test 

con.sql(f"""
SELECT 
  name_1_3,
  buffer_typ,
  longitude,
  latitude,
  year,
  ndvi,
  npp,
  precip,
  taverage,
  tmax,
  aspect,
  elevation,
  ENGTYPE_3,
  Geographic,
  ID_0,
  ID_1,
  ID_2,
  ID_3,
  ISO,
  NAME_0,
  NAME_1,
  NAME_2,
  NAME_3,
  NL_NAME_3,
  pixel_id,
  PriorityCo,
  region,
  second_new,
  slope,
  treated_no,
  Type,
  TYPE_3,
  VARNAME_3,
  woreda,
  zone,
  lc_year,
  land_cover_value,
  polygon_pairs,
  distance_meters
FROM (
  SELECT
    p.name_1_3,
    p.buffer_typ,
    p.longitude,
    p.latitude,
    p.year,
    p.ndvi,
    p.npp,
    p.precip,
    p.taverage,
    p.tmax,
    p.aspect,
    p.elevation,
    p.ENGTYPE_3,
    p.Geographic,
    p.ID_0,
    p.ID_1,
    p.ID_2,
    p.ID_3,
    p.ISO,
    p.NAME_0,
    p.NAME_1,
    p.NAME_2,
    p.NAME_3,
    p.NL_NAME_3,
    p.pixel_id,
    p.PriorityCo,
    p.region,
    p.second_new,
    p.slope,
    p.treated_no,
    p.Type,
    p.TYPE_3,
    p.VARNAME_3,
    p.woreda,
    p.zone,
    p.lc_year,
    p.land_cover_value,
    b.polygon_pairs,
    ST_Distance(p.geometry, b.geometry) AS distance_meters,
    ROW_NUMBER() OVER (PARTITION BY ROWID ORDER BY ST_Distance(p.geometry, b.geometry)) as rn
  FROM (SELECT * FROM '{woreda_points_parquet_utm_file}' LIMIT 1000) AS p
  CROSS JOIN woreda_border_segments_utm AS b
   -- WHERE ST_DWithin(p.geometry, b.geometry, 25000)  -- need to remove to test
) ranked
WHERE rn = 1
""")

## Limited to 1000 points, but why result is 2587 rows?

In [ ]:
# How many rows?

con.sql(f"SELECT COUNT(*) FROM '{woreda_points_parquet_utm_file}'")

In [ ]:
## How many unique points ?

con.sql(f"SELECT COUNT(DISTINCT geometry) FROM '{woreda_points_parquet_utm_file}'")

## 354074

In [ ]:
con.sql(f"""
    SELECT COUNT(DISTINCT (latitude, longitude))
    FROM '{woreda_points_parquet_utm_file}'
""")

- So there are only 350k unique coordinates actually... and ideally, just need to compute distance for these unique points, instead of each row in the 250 million rows table....  
- But can't do this easily, cause there is no unique point id...  

In [ ]:
con.sql(f"SELECT COUNT(DISTINCT pixel_id) FROM '{woreda_points_parquet_utm_file}'")


#### Create a table of unique point id

In [ ]:
# Create table of unique points with custom ID
con.sql(f"""
CREATE OR REPLACE TABLE woreda_unique_points AS
SELECT 
    ROW_NUMBER() OVER () AS custom_point_id,
    latitude,
    longitude
FROM (
    SELECT DISTINCT
        geometry,
        latitude, longitude
    FROM '{woreda_points_parquet_utm_file}'
)
""")

In [ ]:
con.sql("SELECT COUNT(*) FROM woreda_unique_points")

In [ ]:
con.sql("FROM woreda_unique_points LIMIT 3")

In [48]:
## Add geometry column based on latitude, longitude,
## then transform to UTM 37N
## then add columns latitude_utm_37N, longitude_utm_37N


# Add geometry column in WGS84 (EPSG:4326)
con.sql(f"""
CREATE OR REPLACE TABLE woreda_unique_points AS
    SELECT 
        *,
        ST_Point(longitude, latitude) as geometry 
    FROM woreda_unique_points
""")



In [ ]:
con.sql("FROM woreda_unique_points LIMIT 3")

In [50]:
# Transform to UTM 37N (EPSG:32637)

con.sql(f"""
CREATE OR REPLACE TABLE woreda_unique_points AS
SELECT *, ST_Transform(ST_GeomFromText(ST_AsText(geometry)), 'EPSG:4326', 'EPSG:32637', true) AS geometry_utm_37N 
FROM woreda_unique_points 
""")


In [ ]:
con.sql("FROM woreda_unique_points LIMIT 3")

In [52]:

# Extract UTM coordinates into separate columns
con.sql("""
CREATE OR REPLACE TABLE woreda_unique_points AS
SELECT *, ST_X(ST_GeomFromText(ST_AsText(geometry_utm_37N))) AS x_utm_37N, ST_Y(ST_GeomFromText(ST_AsText(geometry_utm_37N))) AS y_utm_37N
FROM woreda_unique_points
""")

In [ ]:
con.sql("FROM woreda_unique_points LIMIT 3")

In [54]:
## Export as .csv

con.sql("COPY (SELECT * FROM woreda_unique_points) TO 'woreda_unique_points.csv' (HEADER, DELIMITER ',')")


#### Real run! (only for unique points)

In [ ]:
con.sql("FROM woreda_unique_points LIMIT 3").df().columns

In [ ]:
## careful big run!
## Option 3 above

## Need to keep all columns in the woreda_points data cause no unique point id, and 
## not sure yet an efficient way of adding columns, without creating temporary copy and then deleting the old table and replacing with new table

con.sql(f"""
CREATE OR REPLACE TABLE woreda_unique_points_distance_to_closest_border_segment AS
SELECT 
  custom_point_id,
   -- border segment id:
  polygon_pairs,
  -- calculated distance  
  distance_meters
FROM (
  SELECT
    p.custom_point_id,  
    -- border segment id:
    b.polygon_pairs,
    -- calculated distance:
    ST_Distance(p.geometry_utm_37N, b.geometry) AS distance_meters, -- point geometry utm !
    ROW_NUMBER() OVER (PARTITION BY p.custom_point_id ORDER BY ST_Distance(p.geometry_utm_37N, b.geometry)) as rn -- point geometry utm !
  FROM (SELECT * FROM woreda_unique_points) AS p
  CROSS JOIN woreda_border_segments_utm AS b
  WHERE ST_DWithin(p.geometry_utm_37N, b.geometry, 25000)  -- point geometry utm ! -- to try is it better without ST_DWithin... 
) ranked
WHERE rn = 1
""")

## 27 m

In [ ]:
## try is it better without ST_DWithin.

con.sql(f"""
CREATE OR REPLACE TABLE woreda_unique_points_distance_to_closest_border_segment__noSTDWithin AS
SELECT 
  custom_point_id,
   -- border segment id:
  polygon_pairs,
  -- calculated distance  
  distance_meters
FROM (
  SELECT
    p.custom_point_id,  
    -- border segment id:
    b.polygon_pairs,
    -- calculated distance:
    ST_Distance(p.geometry_utm_37N, b.geometry) AS distance_meters, -- point geometry utm !
    ROW_NUMBER() OVER (PARTITION BY p.custom_point_id ORDER BY ST_Distance(p.geometry_utm_37N, b.geometry)) as rn -- point geometry utm !
  FROM (SELECT * FROM woreda_unique_points) AS p
  CROSS JOIN woreda_border_segments_utm AS b
  -- WHERE ST_DWithin(p.geometry_utm_37N, b.geometry, 25000)  -- point geometry utm ! --  try is it better without ST_DWithin... 
) ranked
WHERE rn = 1
""")

## 1000m and still running... lol
## killed

In [ ]:
con.sql("SELECT COUNT(*) FROM woreda_unique_points_distance_to_closest_border_segment")

In [ ]:
con.sql("FROM woreda_unique_points_distance_to_closest_border_segment LIMIT 3")

In [60]:
con.sql("""
COPY (SELECT * FROM woreda_unique_points_distance_to_closest_border_segment) 
TO 'woreda_unique_points_distance_to_closest_border_segment.csv' (HEADER, DELIMITER ',')
""")

# Maybe just create distance raster, for each border segment id, mask with the merged buffer

## Border segments

In [ ]:
woreda_border_segments_utm_df = con.sql("SELECT * EXCLUDE geo FROM woreda_border_segments_utm")

## Buffer merged

In [67]:
buffer_merged = gpd.read_file("G:/ales_guyo/guyo_woreda/output/woreda_buffer_25km_all_attrs.shp")

In [ ]:
buffer_merged.shape

In [ ]:
buffer_merged.crs

In [70]:
buffer_merged_utm = buffer_merged.to_crs("EPSG:32637")

In [ ]:
buffer_merged_utm.explore()

## 1-km template raster

In [29]:
import rasterio
import rioxarray
import numpy as np

In [79]:
## create a 1-km raster in EPSG:32637, within the bbox of buffer_merged_utm

# Get bounds from buffer_merged_utm
bounds = buffer_merged_utm.total_bounds

# Round bounds to nearest km
xmin = np.floor(bounds[0]/1000)*1000
ymin = np.floor(bounds[1]/1000)*1000 
xmax = np.ceil(bounds[2]/1000)*1000
ymax = np.ceil(bounds[3]/1000)*1000

# Calculate dimensions
width = int((xmax - xmin) / 1000)
height = int((ymax - ymin) / 1000)

# Create transform
transform = rasterio.transform.from_origin(xmin, ymax, 1000, 1000)

# Create empty raster array
template_raster = np.random.rand(height, width)

# Create metadata for the raster
meta = {
    'driver': 'GTiff',
    'dtype': 'float32',
    'nodata': None,
    'width': width,
    'height': height,
    'count': 1,
    'crs': 'EPSG:32637',
    'transform': transform
}

# Save the template raster
with rasterio.open('template_1km_random_ok.tif', 'w', overwrite=True, **meta) as dst:
    dst.write(template_raster.astype('float32'), 1)


In [45]:
# import rasterio
# import numpy as np

# # Open the template raster and keep the dataset object
# template_ds = rasterio.open("template_1km_random_ok.tif")
# # Get the metadata
# template_meta = template_ds.meta
# # Read the raster data
# template_raster = template_ds.read(1)

# template_ds.close()

In [80]:
template_raster_da = rioxarray.open_rasterio("template_1km_random_ok.tif")

In [ ]:
template_raster_da

In [81]:
# Crop the template raster to the buffer_merged_utm geometry
template_raster_da_cropped = template_raster_da.rio.clip(buffer_merged_utm.geometry, 
                                                     buffer_merged_utm.crs,
                                                     drop=True)

# Save the cropped raster
template_raster_da_cropped.rio.to_raster("template_1km_random_ok_cropped.tif")


In [ ]:
## plot interactive map

import leafmap.foliumap as leafmap

bounds = template_raster_da_cropped.rio.bounds()
center_lat = (bounds[1] + bounds[3]) / 2
center_lon = (bounds[0] + bounds[2]) / 2

m = leafmap.Map(center=[center_lat, center_lon], zoom=10)

# Add the raster layer
m.add_raster(template_raster_da_cropped, layer_name="Template Raster", colormap="viridis")

# Display the map
m



In [ ]:
template_raster_da_cropped.plot()

## Rasterize the buffer_merged_utm

In [ ]:
# # Rasterize the buffer using geocube
# from geocube.api.core import make_geocube

# # Create geocube from buffer geometry
# buffer_raster = make_geocube(
#     vector_data=buffer_merged_utm,
#     like=template_raster_da_cropped,
#     fill=0,
#     dtype='uint8'
# )


## Rasterize the border segments

In [ ]:
# bounds = california_borders.total_bounds
# res = 0.5
# transform = rasterio.transform.from_origin(
#     west=bounds[0], 
#     north=bounds[3], 
#     xsize=res, 
#     ysize=res
# )
# rows = math.ceil((bounds[3] - bounds[1]) / res)
# cols = math.ceil((bounds[2] - bounds[0]) / res)
# shape = (rows, cols)
# shape

In [ ]:
# california_raster1 = rasterio.features.rasterize(
#     [(g, 1) for g in california_borders],
#     out_shape=shape,
#     transform=transform,
#     all_touched=True,
#     fill=np.nan,
#     dtype=np.float64
# )

## Create distance rasters

In [ ]:
## Option 3: Using GDAL proximity (most efficient)


# import subprocess
# import os

# # Get unique border segment IDs
# border_segment_ids = lines_gdf['border_segment_id'].unique()

# for segment_id in border_segment_ids:
#     # Filter the line for this segment
#     line_segment = lines_gdf[lines_gdf['border_segment_id'] == segment_id]
    
#     # Save line segment to temporary shapefile
#     temp_line_file = f"temp_line_{segment_id}.shp"
#     line_segment.to_file(temp_line_file)
    
#     # Rasterize the line segment
#     temp_raster_file = f"temp_raster_{segment_id}.tif"
#     rasterize_cmd = f"""
#     gdal_rasterize -burn 1 -tr {template_raster_da_cropped.rio.resolution()[0]} {template_raster_da_cropped.rio.resolution()[1]} \
#     -te {template_raster_da_cropped.rio.bounds()[0]} {template_raster_da_cropped.rio.bounds()[1]} \
#     {template_raster_da_cropped.rio.bounds()[2]} {template_raster_da_cropped.rio.bounds()[3]} \
#     -ot Byte -of GTiff -a_nodata 0 {temp_line_file} {temp_raster_file}
#     """
#     subprocess.run(rasterize_cmd, shell=True)
    
#     # Calculate proximity
#     output_filename = f"distance_to_segment_{segment_id}.tif"
#     proximity_cmd = f"""
#     gdal_proximity.py {temp_raster_file} {output_filename} \
#     -distunits GEO -values 1 -nodata 0 -ot Float32
#     """
#     subprocess.run(proximity_cmd, shell=True)
    
#     # Clean up temporary files
#     os.remove(temp_line_file)
#     os.remove(temp_raster_file)
    
#     print(f"Created distance raster for segment {segment_id}")
